In [1]:
# Bagian 1: Setup dan Pengolahan Data

import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv("heart.csv")

# Split features dan target
X = data.drop(columns=['target'])
y = data['target']

# Normalize features untuk performa MLP yang lebih baik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Bagi data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Konversi data menjadi PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Target untuk klasifikasi
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [2]:
# Bagian 2: Definisi Model MLP

import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Definisikan fungsi untuk membuat DataLoader
def create_data_loader(X, y, batch_size):
    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Definisikan model MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLP, self).__init__()
        layers = []
        current_size = input_size
        for layer_size in hidden_layers:
            layers.append(nn.Linear(current_size, layer_size))
            layers.append(activation_fn)
            current_size = layer_size
        layers.append(nn.Linear(current_size, 2))  # Output layer untuk klasifikasi biner
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [3]:
# Bagian 3: Fungsi Pelatihan dan Evaluasi

import torch.optim as optim

# Fungsi untuk melatih dan mengevaluasi model
def train_and_evaluate_model(hidden_layers, activation_fn, learning_rate, batch_size, epochs):
    # Buat DataLoader
    train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size)
    test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size)

    # Inisialisasi model, loss function, dan optimizer
    model = MLP(input_size=13, hidden_layers=hidden_layers, activation_fn=activation_fn)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Loop pelatihan
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Reset gradien
            predictions = model(X_batch)  # Forward pass
            loss = criterion(predictions, y_batch)  # Hitung loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update bobot

    # Evaluasi
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            predictions = model(X_batch)
            _, predicted = torch.max(predictions, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    return accuracy


In [4]:
# Bagian 4: Eksperimen Hyperparameter

import itertools

# Hyperparameter settings
hidden_layer_options = [[4], [8], [16], [32], [64]]
activation_functions = [nn.Identity(), nn.Sigmoid(), nn.ReLU(), nn.Softmax(dim=1), nn.Tanh()]
learning_rates = [10, 1, 0.1, 0.01, 0.001]
batch_sizes = [16, 32, 64, 128, 256]
epoch_options = [1, 10, 25, 50, 100]

# Simpan hasil
results = []
for hidden_layers, activation_fn, lr, batch_size, epochs in itertools.product(
    hidden_layer_options, activation_functions, learning_rates, batch_sizes, epoch_options):
    try:
        accuracy = train_and_evaluate_model(
            hidden_layers=hidden_layers,
            activation_fn=activation_fn,
            learning_rate=lr,
            batch_size=batch_size,
            epochs=epochs
        )
        results.append({
            'Hidden Layers': hidden_layers,
            'Activation Function': activation_fn.__class__.__name__,
            'Learning Rate': lr,
            'Batch Size': batch_size,
            'Epochs': epochs,
            'Accuracy': accuracy
        })
    except Exception as e:
        # Catat error
        results.append({
            'Hidden Layers': hidden_layers,
            'Activation Function': activation_fn.__class__.__name__,
            'Learning Rate': lr,
            'Batch Size': batch_size,
            'Epochs': epochs,
            'Accuracy': None,
            'Error': str(e)
        })

# Simpan hasil ke DataFrame
results_df = pd.DataFrame(results)

# Simpan ke file CSV
results_df.to_csv('mlp_hyperparameter_results.csv', index=False)
print("Eksperimen selesai. Hasil disimpan dalam 'mlp_hyperparameter_results.csv'.")


Eksperimen selesai. Hasil disimpan dalam 'mlp_hyperparameter_results.csv'.


In [5]:
# Bagian 5: Analisis Hasil

# Load hasil dari CSV
results_df = pd.read_csv('mlp_hyperparameter_results.csv')

# Tampilkan 10 kombinasi terbaik berdasarkan akurasi
top_results = results_df.sort_values(by='Accuracy', ascending=False).head(10)
print(top_results)


     Hidden Layers Activation Function  Learning Rate  Batch Size  Epochs  \
2424          [32]                Tanh            1.0         256     100   
1808          [16]                Tanh            0.1          32      50   
1819          [16]                Tanh            0.1         128     100   
1824          [16]                Tanh            0.1         256     100   
1438          [16]             Sigmoid            0.1          64      50   
1434          [16]             Sigmoid            0.1          32     100   
1433          [16]             Sigmoid            0.1          32      50   
1427          [16]             Sigmoid            0.1          16      25   
2433          [32]                Tanh            0.1          32      50   
3049          [64]                Tanh            1.0         256     100   

      Accuracy  
2424       1.0  
1808       1.0  
1819       1.0  
1824       1.0  
1438       1.0  
1434       1.0  
1433       1.0  
1427       1.0  